# Childrens Act Disposals

In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [ ]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
# create path for within athena FCSQ database in the S3 folder
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_disps/"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2022-02-21"

#### Extracting disposals 

In [ ]:
# The first select query extracts orders where the type of order is defined by the event model.
# The second select query extracts disposals where the type of order is determined by the value of the field model. Included in this are orders of no order (ORDNOM) and orders refused (ORDREF).
    # C43 contains child arrangement orders (contact/residence - the most common orders types in children act cases) 
# The cross join unnest function in the second select query splits the value (order types - where multiple values can be entered) into one row per value
pydb.create_temp_table(
f""" 
SELECT 
  event,
  case_number,
  event_model,
  event_model as lookup_event_code,
  receipt_date,
  entry_date,
  COALESCE(receipt_date, entry_date) AS disp_date,
  creating_court,
  CAST(NULL AS VARCHAR) AS field_model,
  CAST(NULL AS VARCHAR) AS all_event_ord_types,
  'N/A' AS order_type
FROM 
  {db1}.events 
WHERE 
  ((event_model IN ('C23','C25','C26','C27','C28','C29',
                    'C30','C31','C32A','C32B','C33','C34A','C34B','C35A','C35B','C37',
                    'C38A','C38B','C39','C40','C42','C43A','C44A','C44B','C45A','C45B',
                    'C53','C80','C81','C82') 
            AND error = 'N')
    OR (event_model = 'CPA' AND error = 'N' AND narrative Is Null))
  AND mojap_snapshot_date = date '{snapshot_date}'

UNION ALL

SELECT 
  e.event,
  e.case_number,
  e.event_model,
  CASE WHEN e.event_model IN ('C43', 'ORDNOM', 'ORDREF')
      THEN 'GEN'
      ELSE e.event_model END
    AS lookup_event_code,
  e.receipt_date,
  e.entry_date,
  COALESCE(e.receipt_date, e.entry_date) AS disp_date,
  e.creating_court,
  f.field_model,
  f.value as all_event_ord_types,
  TRIM(ord_type) as order_type
FROM 
  {db1}.events e
  INNER JOIN {db1}.event_fields f
    ON e.event = f.event
  CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(ord_type)  
WHERE 
  ((f.field_model IN ('C21_3', 'C24_6', 'C36_5', 'C43_5', 'ORDNOM_5', 'ORDREF_5'))
    AND error = 'N')
  AND e.mojap_snapshot_date = date '{snapshot_date}'
  AND f.mojap_snapshot_date = date '{snapshot_date}'
  
""",

"ca_disps1")

#### Withdrawn events
###### Withdrawn events are extracted seperately to other disposal events as they need to be linked back to the original application to get the order type being withdrawn

In [ ]:
#Extracting withdrawn events (G63). 
#To get the type of order being withdrawn we can link back to the original application (U22/G50) event
#Cross Join unnest function splits out the application values into a single row per order type

pydb.create_temp_table(
f""" 
SELECT 
  w.event,
  e.case_number,
  e.event_model,
  'GEN' AS lookup_event_code,
  e.receipt_date,
  e.entry_date,
  COALESCE(e.receipt_date, e.entry_date) AS disp_date,
  e.creating_court,
  w.field_model,
  a.value as all_event_ord_types,
  TRIM(ord_type) as order_type
FROM 
  {db1}.event_fields w
  left join {db1}.event_fields a
    on w.value = CAST (a.event AS VARCHAR)
  left join {db1}.events e
    on w.event = e.event
CROSS JOIN UNNEST(SPLIT(a.value,',')) AS t(ord_type)    
WHERE 
  w.field_model = 'G63_1' 
  AND a.field_model IN ('G50_AT','U22_AT')
  AND e.error = 'N'
  AND w.mojap_snapshot_date = date '{snapshot_date}'
  AND e.mojap_snapshot_date = date '{snapshot_date}'
  AND a.mojap_snapshot_date = date '{snapshot_date}'
  
""",

"ca_withdrawn")

#### Joining withdrawn events to other disposal events

In [ ]:
#Append withdrawn events to the main set of disposal events
pydb.create_temp_table(
f""" 
SELECT 
  *
FROM
  __temp__.ca_disps1
UNION ALL 
SELECT
  *
FROM
  __temp__.ca_withdrawn

""",
    
"ca_disps_all")

#### Adding the order & disposal type description and case type, filtering out order types we do not include

In [ ]:
# The order type lookup file has been created previosuly
# Order code to be up to 45 but including up to 54 as this is what some of the published data includes
pydb.create_temp_table(
f"""
SELECT
  t1.*,
  CASE WHEN t2.order_code < 19 OR (SUBSTR(case_number,5,1) = 'C') THEN 'C'
        ELSE 'P' END 
      AS case_type,
  CASE WHEN t1.event_model = 'G63' THEN 3
       WHEN t1.event_model = 'ORDREF' THEN 4
       WHEN t1.event_model = 'ORDNOM' THEN 5
       WHEN t2.order_code BETWEEN 40 AND 45 THEN 2
       ELSE 1 END
    AS disp_type_code,
  CASE WHEN t1.event_model = 'G63' THEN 'Withdrawn'
       WHEN t1.event_model = 'ORDREF' THEN 'Refused'
       WHEN t1.event_model = 'ORDNOM' THEN 'No order'
       WHEN t2.order_code BETWEEN 40 AND 45 THEN 'Interim Order'
       ELSE 'Order' END
    AS disp_type,
  order_code,
  order_desc
FROM
  __temp__.ca_disps_all t1
  LEFT JOIN {fcsq_db}.ca_order_type_lookup t2
    ON t1.lookup_event_code = t2.event_model
    AND t1.order_type = t2.value
WHERE
  t2.order_code BETWEEN 0 AND 54

""",
    
"ca_disps_order_name")

### Adding children to the disposal events
##### This is done in 3 stages:    
######   - First, some children are recorded directly on the event
######   - Second, where the child data field within the event is blank, we pull out all the children recorded on the case from the roles table
######   - Finally, there are a small number of cases in which children are neither recorded on the event or the case. Here we default to one child per disposal event, with unknown characteristics

#### Children - event level

In [ ]:
#First we extract the child role ID where the child is listed on the event
#Cross join unnest function splits where multiple children are listed in the event field into one row per child
pydb.create_temp_table(
f"""
  SELECT 
    a.*,
    f.value children,
    TRY_CAST(TRIM(child_role_id) as bigint) child_role_id
  FROM 
    __temp__.ca_disps_order_name a
    LEFT JOIN {db1}.event_fields f
      ON f.event = a.event
   CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(child_role_id)
  WHERE 
     f.field_model IN('C21_CH','C23_CH','C24_CH','C25_CH','C26_CH','C27_CH','C28_CH','C29_CH',
                      'C30_CH','C31_CH','C32A_CH','C32B_CH','C33_CH','C34A_CH',
                      'C34B_CH','C35A_CH','C35B_CH','C36_CH','C37_CH','C38A_CH',
                      'C38B_CH','C39_CH','C40_CH','C42_CH','C43_CH',
                      'C43A_CH','C44A_CH','C44B_CH','C45A_CH','C45B_CH','C53_CH',
                      'C80_CH','C81_CH','C82_CH','CPA_CH','ORDNOM_CH','ORDREF_CH', 'G63_CH')
    AND child_role_id <> ''
    AND f.mojap_snapshot_date = DATE'{snapshot_date}'
""",

"ca_disps_child_event")

#### Children - case level

In [ ]:
#Secondly, we take all the events in which children were not directly recorded and link to the case details to get child information recorded on the case
#Where a child id is not recorded on the event, this means the event relates to all children listed on the case
#We get the dob and gender of the child from the party table
pydb.create_temp_table(
f"""
  SELECT
    a.*,
    r.role child_role_id,
    p.dob,
    p.gender,
    r.delete_flag
  FROM
    __temp__.ca_disps_order_name a
    JOIN {db1}.roles r 
      on a.case_number = r.case_number
    JOIN {db1}.parties p 
      on r.party = p.party 
  WHERE
    event not in (SELECT event FROM __temp__.ca_disps_child_event)
    AND role_model in ('CHLDC', 'CHLDZ')
    AND r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
""",

"ca_disps_child_case")

#### Children - not recorded on event or case

In [ ]:
#There are a small number of disposals where the children are not reocorded on the event and there are also no children recorded on the case. This tends to be for DV cases, where CA order types 
#   may have been made but children are not necessarily recorded on the case. Here we default to just one child per order/disposal, with unknown characteristics.
pydb.create_temp_table(
f"""
  SELECT
    *,
    CAST(NULL AS INTEGER) AS child_role_id,
    CAST(NULL AS DATE) dob,
    CAST(NULL AS INTEGER) AS gender
  FROM
    __temp__.ca_disps_order_name
  WHERE
    event NOT IN (SELECT event FROM __temp__.ca_disps_child_event)
    AND event NOT IN (SELECT event FROM __temp__.ca_disps_child_case)
""",

"ca_disps_no_child")

### Child disposals data set
##### Combining children data sets together

In [ ]:
#Append the two datasets created previouly, which also includes all disposal data
#The child event table also links to the party table to get the dob/gender info

pydb.create_temp_table( 
f"""
  SELECT 
    a.case_number,
    a.disp_date,
    EXTRACT(YEAR FROM a.disp_date) AS Year,
    EXTRACT(QUARTER FROM a.disp_date) AS Quarter,
    a.case_type,
    a.receipt_date,
    a.entry_date,
    a.event,
    a.event_model,
    a.field_model,
    a.order_type,
    a.disp_type_code,
    a.disp_type,
    a.order_code,
    a.order_desc,
    a.child_role_id,
    p.dob,
    p.gender,
    a.creating_court
  FROM
    __temp__.ca_disps_child_event a
    JOIN {db1}.roles r on a.child_role_id = r.role
    JOIN {db1}.parties p on r.party = p.party
  WHERE
    r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
    AND r.delete_flag = 'N'
  UNION ALL
  SELECT
    a.case_number,
    a.disp_date,
    EXTRACT(YEAR FROM a.disp_date) AS Year,
    EXTRACT(QUARTER FROM a.disp_date) AS Quarter,
    a.case_type,
    a.receipt_date,
    a.entry_date,
    a.event,
    a.event_model,
    a.field_model,
    a.order_type,
    a.disp_type_code,
    a.disp_type,
    a.order_code,
    a.order_desc,
    a.child_role_id,
    a.dob,
    a.gender,
    a.creating_court
  FROM
    __temp__.ca_disps_child_case a
  WHERE
    a.delete_flag = 'N'
  UNION ALL
  SELECT
    a.case_number,
    a.disp_date,
    EXTRACT(YEAR FROM a.disp_date) AS Year,
    EXTRACT(QUARTER FROM a.disp_date) AS Quarter,
    a.case_type,
    a.receipt_date,
    a.entry_date,
    a.event,
    a.event_model,
    a.field_model,
    a.order_type,
    a.disp_type_code,
    a.disp_type,
    a.order_code,
    a.order_desc,
    a.child_role_id,
    a.dob,
    a.gender,
    a.creating_court
  FROM
    __temp__.ca_disps_no_child a
""",

"ca_disps_all_children")

### Removal of duplicate children

#### This code is not currently used as the old SAS process never removed duplicate orders made - but placing here as it's likely we will come to this in future

In [ ]:
pydb.create_temp_table(
f"""
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY case_number, child_role_id, order_code, receipt_date, disp_type 
                       ORDER BY case_number, child_role_id, order_code, receipt_date, event) 
      AS dup_rank
  FROM
    __temp__.ca_disps_all_children

""",
    
"ca_disp_dup_flag")

In [ ]:
#get an idea of how many might be duplicate order types
pydb.read_sql_query ("select count(*) count from __temp__.ca_disp_dup_flag where year > 2010 and disp_type_code = 1 and dup_rank > 1")

In [ ]:
#drop table in Athena (if it already exists)
_ = pydb.start_query_execution_and_wait(f"""DROP TABLE {fcsq_db}.ca_disps""")

In [ ]:
#set S3 file path
disp_s3_path = os.path.join(fcsq_db_path, "disps/")
# Delete all the underlying data stored within the S3 location
if wr.s3.list_objects(disp_s3_path):
    print("deleting objs")
    wr.s3.delete_objects(disp_s3_path)
    
t_disps = f"""
CREATE TABLE {fcsq_db}.ca_disps WITH
(
    external_location='{disp_s3_path}'
) AS   
SELECT
*
FROM
__temp__.ca_disps_children_FO
"""
    
_ = pydb.start_query_execution_and_wait(t_disps)    